In [1]:
# Import modules
!pip install folium
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests 
from pandas.io.json import json_normalize 
import folium

# DATA

Import API Foursquare keys

In [2]:
CLIENT_ID = '' 
CLIENT_SECRET = '' 
VERSION = '20210118'

Call the url

In [3]:
LIMIT = 500 
cities = ['Los Angeles', 'New York', 'London', 'Milan', 'Barcelona']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        '4d4b7105d754a06376d81259') 
    results[city] = requests.get(url).json()

In [4]:
df_venues={}
for city in cities:
    venues = pd.json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

Define maps and display total number of bar in each city

In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.Marker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.6,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of nightlife spots {city} = ", results[city]['response']['totalResults'])

Total number of nightlife spots Los Angeles =  135
Total number of nightlife spots New York =  193
Total number of nightlife spots London =  118
Total number of nightlife spots Milan =  101
Total number of nightlife spots Barcelona =  111
